In [6]:
!pip install -r requirements.txt

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.3-cp312-cp312-macosx_10_13_universal2.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)
Using cached charset_normalizer-3.4.3-cp312-cp312-macosx_10_13_universal2.whl (205 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)

[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [36]:
import os
import requests
import pandas as pd
import re
import brotli
import json


###############################################
############### GET_ALL_YEARS #################

# ---------- HEADERS ----------
HEADERS = {
    "accept": "application/json",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "en-US,en;q=0.9",
    "dnt": "1",
    "priority": "u=1, i",
    "referer": "https://ifsc.results.info/event/1442/",
    "sec-ch-ua": '"Chromium";v="139", "Not;A=Brand";v="99"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/139.0.0.0 Safari/537.36"
    ),
    "x-csrf-token": "ARqeGlAj3ZQZxLJKwKbeXcPdnp4Jj6cBbXQZ9dJdhXklQVDJDhr9f0Ro9PbNQTUiZoakGPC0Sq3BH-zF2EVluQ",
}

# ---------- COOKIES ----------
COOKIES_STRING = (
    "_ga=GA1.2.730144941.1748434369; _verticallife_resultservice_session=aPUXmlTgFQfglpASE1pDEtGUO5KKhUigQqQGpOgiE5h2W1eRjWuAD95thCgJaTVMqZoeJIH7M%2BrgIn9DuqMkUkeP4nnvxrdjIjCJOm2DJ9QDTRn1LQIu8TBzTTyEQa%2BrL4RiCQPMTDOfcW5peS10oNG%2BXp%2F3RrhUy6dTrw%2FKABsdZzmmAYx4tJM8R%2BZJqnDPL9rVNEQhoEX9PzG1FoCcJgz4HQK59yJVtd8DR1MascKHy2etGoruWbB1%2Bv9jmXag2cwOkvqSxkixq%2BJHL7%2F0Md8coEWVvhpoi0Vwz%2FcD4llWqLL7FuQVTmoT6w%3D%3D--EeDuSer%2F54c0V9xb--9sfg9uHTw1RE%2Fa0pKUwyEQ%3D%3D; _gid=GA1.2.565234169.1756491233; _gat_UA-157153964-4=1; _ga_3CW645GZWB=GS2.2.s1756491233$o1$g0$t1756491233$j60$l0$h0"
)

COOKIES = {key: value for part in COOKIES_STRING.split("; ") if "=" in part for key, value in [part.split("=", 1)]}

# ---------- API HELPER ----------
BASE_API = "https://ifsc.results.info/api/v1/"

def get_api_data(endpoint: str) -> dict:
    """
    Fetch and decode JSON data from an IFSC API endpoint, handling Brotli compression.
    If endpoint is relative (e.g., '/seasons'), it is joined with BASE_API.
    """
    url = endpoint if endpoint.startswith("http") else BASE_API + endpoint.lstrip("/")
    print(url)
    response = requests.get(url, headers=HEADERS, cookies=COOKIES)

    # Raise an exception for HTTP errors (4xx or 5xx)
    response.raise_for_status()

    # Check for Brotli compression and decompress if necessary
    if response.headers.get('Content-Encoding') == 'br':
        try:
            # Decompress the content before attempting to decode as JSON
            decompressed_content = brotli.decompress(response.content)
            # The decompressed content is bytes, so we need to decode it to a string
            json_text = decompressed_content.decode('utf-8')
        
        except UnicodeDecodeError as e:
            print(f"Unicode decode error: {e}")
            return {}
    else:
        # If not Brotli, use the standard response text
        json_text = response.text


    try:
        # Attempt to parse the JSON text
        return json.loads(json_text)
    except json.JSONDecodeError as err:
        print(f"JSON Decode Error: {err} from URL: {url}")
        print("Response text was:", json_text)
        return {}



def get_worldcup_leagues(season_api="/api/v1/"):
    seasons = get_api_data(season_api)['seasons']
    results = []
    for s in seasons:
        year = s['name']
        for league in s['leagues']:

            if 'World Cups' in league['name'] and 'Youth' not in league['name']:
                results.append({
                    "year": int(year),
                    "league_name": league['name'],
                    "url": league['url']
                    })
    return pd.DataFrame(results)

if __name__ == "__main__":
    # You might need to install the brotli library: pip install brotli
    data = get_api_data("info")
    print(data.keys())
    

https://ifsc.results.info/api/v1/info
dict_keys(['logo_url', 'logo_link', 'federation'])


In [29]:
import brotli

try:
    # Decompress the brotli content
    decompressed_content = brotli.decompress(response.content)
    # Decode the bytes to a string
    json_string = decompressed_content.decode('utf-8')
    # Parse the JSON string
    data = json.loads(json_string)
    print(data)
except brotli.error as e:
    print(f"Not brotli-compressed content: {e}")

{'id': 10236, 'event': 'IFSC Youth European Championships Žilina 2025', 'event_id': 1442, 'dcat_id': 818, 'discipline': 'Lead', 'status': 'active', 'status_as_of': '2025-08-29 16:27:58 UTC', 'category': 'U21 Women', 'round': 'Semi-Final', 'format': 'IFSC: 1 route', 'routes': [{'id': 17294, 'name': '1', 'startlist': '/api/v1/routes/17294/startlist', 'ranking': '/api/v1/routes/17294/results'}], 'startlist': [{'athlete_id': 13001, 'route_start_positions': [{'route_name': '1', 'route_id': 17294, 'position': 1}], 'name': 'EL Zeli̇ha', 'firstname': 'Zeli̇ha', 'lastname': 'EL', 'bib': '464', 'country': 'TUR', 'flag_url': 'https://d1n1qj9geboqnb.cloudfront.net/flags/TUR.png', 'federation_id': 62}, {'athlete_id': 12331, 'route_start_positions': [{'route_name': '1', 'route_id': 17294, 'position': 2}], 'name': 'BERNARDOVA Barbora', 'firstname': 'Barbora', 'lastname': 'BERNARDOVA', 'bib': '318', 'country': 'CZE', 'flag_url': 'https://d1n1qj9geboqnb.cloudfront.net/flags/CZE.png', 'federation_id': 1